In [3]:
import time
import numpy as np
from itertools import combinations_with_replacement
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle
from itertools import combinations_with_replacement
from sympy.core import Mul
import copy

In [4]:
data = pd.read_csv('glass.csv')

In [5]:
data

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,7
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,7
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,7
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,7


In [6]:
Y = data.loc[:, 'Type']
X = data.iloc[:, 0:-1]

In [7]:
X_train, X_test, Y_train, Y_test = tts(X, Y, test_size = 0.4, random_state = 42)

In [8]:
def Multiply(a):
    ans = 1
    for i in a:
        ans*=i
    return ans

In [9]:
def Create_features(variables):
    ans = []
    for i in range(1,5):
        l = list(combinations_with_replacement(variables, i))
        for j in l:
            ans.append(Multiply(j))
            #print(Multiply(j))
    ans.append(1)
    ans = np.array(ans).astype(float)
    return ans.reshape((ans.shape[0],1)) 

In [10]:
def smo(X,Y, C=0.1, toler=0.001, maxIter=5):
    #dataMatrix = mat(dataMatIn); 
    #labelMat = mat(classLabels).transpose()
    b = 0; m = X.shape[0]
    alpha=np.zeros(shape=(X.shape[0],1))
    #alphas = mat(zeros((m,1)))
    iter = 0
    while (iter < maxIter):
        alphaPairsChanged = 0
        for i in range(m): #Fetch one piece of data each time, corresponding to alpha1
            fXi = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b #Calculate the classification function result, directly apply the classification function formula (8)
            Ei = fXi-float(labelMat[i])#if checks if an example violates KKT conditions #The difference between the calculated result and the actual classification
            if ((labelMat[i]*Ei < -toler) and (alphas[i] < C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
                               #3 cabinets that do not meet the KKT condition in the iteration are used here, ((labelMat[i]*Ei> toler) and (alphas[i]> 0) as an example, assuming this is a positive classification, then (labelMat[i ]*Ei> toler)
                               # Ei>0 and Ei> toler, because labelMat[i]=1, yi*fXi>1 is obtained, at this time, alpha1>0 is not satisfied, and the original alpha1=0
                j = selectJrand(i,m)
                fXj = float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)) + b # alpha2 random selection
                Ej = fXj - float(labelMat[j])
                alphaIold = alphas[i].copy(); alphaJold = alphas[j].copy();
                if (labelMat[i] != labelMat[j]): # y1 and y1 have different signs, calculate L and H
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L==H: 
                    print ("L==H") 
                    continue
                    eta = 2.0 * dataMatrix[i,:]*dataMatrix[j,:].T-dataMatrix[i,:]*dataMatrix[i,:].T-dataMatrix[j,:]*dataMatrix[j,:]. T #Calculate η
                if eta >= 0: 
                    print ("eta>=0") 
                    continue #When calculating η =0, divide by 0, skip this case temporarily
                    alphas[j] -= labelMat[j]*(Ei-Ej)/eta #Calculate alpha2, refer to formula (5)
                    alphas[j] = clipAlpha(alphas[j],H,L)
                if (abs(alphas[j]-alphaJold) <0.00001):
                    print ("j not moving enough")
                    continue #reach the optimization threshold
                    alphas[i] += labelMat[j]*labelMat[i]*(alphaJold - alphas[j])#update i by the same amount as j
                                                                                                                                                 #the update is in the oppostie direction Update alpha1, formula (6)
                                 #Calculate b1,b2, formula (7)                                                        
                    b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                    b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T - labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                    if (0 < alphas[i]) and (C > alphas[i]):
                        b = b1
                    elif (0 < alphas[j]) and (C > alphas[j]): 
                        b = b2
                    else: 
                        b = (b1 + b2)/2.0
                        alphaPairsChanged += 1
                        print ("iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged))
                if (alphaPairsChanged == 0): 
                    iter += 1
                else: 
                    iter = 0
        print ("iteration number: %d" % iter)
         
        return b,alphas   

In [11]:
def predict(X,Y,alpha,b,x,sigma):
    result=0.0

    for i in range(X.shape[0]):
        result+=(alpha[i]*Y[i]*(X[i,:] , x,sigma));

    result+=b

    return result

In [12]:
def compute_cost(W, X, Y):
    # calculate hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = regularization_strength * (np.sum(distances) / N)
    # calculate cost
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost

In [13]:
def remove_correlated_features(X):
    corr_threshold = 0.9
    corr = X.corr()
    drop_columns = np.full(corr.shape[0], False, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i + 1, corr.shape[0]):
            if corr.iloc[i, j] >= corr_threshold:
                drop_columns[j] = True
    columns_dropped = X.columns[drop_columns]
    X.drop(columns_dropped, axis=1, inplace=True)
    return columns_dropped

In [14]:
def computeW(alphas, X, Y):
    m,n = shape(X)
    w = zeros((n,1))
    for i in range(m):
        w += multiply(alphas[i]*Y[i],X[i,:].T)
        return w

In [ ]:
def sgd(features, outputs):
    max_epochs = 5000
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs):
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)

        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is: {} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights